## 函数映射apply
* Seris每个单独元素应用于函数
* 沿DataFrame的轴(index或columns)应用与函数

In [3]:
import  numpy as np
import  pandas as pd
'''
func : function
    Function to apply to each column or row.
    
axis : {0 or 'index', 1 or 'columns'}, default 0
    Axis along which the function is applied:

    * 0 or 'index': apply function to each column.
    * 1 or 'columns': apply function to each row.

result_type : {'expand', 'reduce', 'broadcast', None}, default None
    These only act when ``axis=1`` (columns):

    * 'expand' : list-like results will be turned into columns.
    * 'reduce' : returns a Series if possible rather than expanding
      list-like results. This is the opposite of 'expand'.
    * 'broadcast' : results will be broadcast to the original shape
      of the DataFrame, the original index and columns will be
      retained.

    The default behaviour (None) depends on the return value of the
    applied function: list-like results will be returned as a Series
    of those. However if the apply function returns a Series these
    are expanded to columns.

**kwds
    Additional keyword arguments to pass as keywords arguments to
    `func`.
'''
s = pd.Series([20, 21, 12],
               index=['London', 'New York', 'Helsinki'])
s

London      20
New York    21
Helsinki    12
dtype: int64

In [4]:
def square(x):
     return x ** 2

def classify(x):
    if abs(x) > 16:
        return 1
    else:
        return 0

In [5]:
s.apply(square) # 默认axis=0   

London      400
New York    441
Helsinki    144
dtype: int64

In [6]:
s = s.apply(classify)

In [7]:
s

London      1
New York    1
Helsinki    0
dtype: int64

In [8]:
s.apply('I am a {}'.format)

London      I am a 1
New York    I am a 1
Helsinki    I am a 0
dtype: object

In [9]:
df = pd.DataFrame([[4, 9],] * 3, columns=['A', 'B'])
df

,A,B
0,4,9
1,4,9
2,4,9


In [11]:
df.apply(np.sqrt)

,A,B
0,2.0,3.0
1,2.0,3.0
2,2.0,3.0


In [12]:
df.apply(np.sum, axis=0)

A    12
B    27
dtype: int64

In [13]:
df.apply(lambda x: [1, 2], axis=1) # 使用匿名函数

0    [1, 2]
1    [1, 2]
2    [1, 2]
dtype: object

In [14]:
# Passing result_type=’expand’ will expand list-like results to columns of a Dataframe
df.apply(lambda x: [1, 2], axis=1, result_type='expand') # 扩展为列

,0,1
0,1,2
1,1,2
2,1,2


In [21]:
# Returning a Series inside the function is similar to passing result_type='expand'. The resulting column names will be the Series index.
df.apply(lambda x: pd.Series([1, 2, 3], index=['foo', 'bar', 'tom']), axis=1) # 返回值为Series,自动扩展为列

,foo,bar,tom
0,1,2,3
1,1,2,3
2,1,2,3


In [24]:
# Passing result_type='broadcast' will ensure the same shape result
df.apply(lambda x: [1, 2], axis=1, result_type='broadcast') # 形状不变,列名也不变

,A,B
0,1,2
1,1,2
2,1,2


In [14]:
def f(x):
    """
    计算DataFrame每一列的最大和最小值
    :param x: DataFrame的每一列
    :return: Series
    """
    return pd.Series([min(x), x.max()], index=['min', 'max'])

df.apply(f, axis=0)

,A,B
min,4,9
max,4,9


In [76]:
def g(x, func1=np.mean):
    """
    对DataFrame函数操作
    :param x: DataFrame的每一列
    :param func1: 要执行的函数
    :return: 
    """
    return func1(x)

In [77]:
df.apply(func=g, func1=np.sqrt, axis=0) # 向apply中的函数(g(x))传递参数

,A,B
0,2.0,3.0
1,2.0,3.0
2,2.0,3.0


In [78]:
def get_stats(group, gfunc=np.sum): # apply中同时应用多个函数(可传参)
    if group.name == 'A': # 不同列应用不同的函数
        return pd.Series([group.min(), group.max(), np.ptp(group), g(group, func1=gfunc)], 
                     index=['min', 'max', 'ptp', 'g'])
    else:
        return pd.Series([group.min(), group.max(), group.mean(), g(group, func1=gfunc)], 
                     index=['min', 'max', 'mean',  'g'])

In [79]:
df.apply(func=get_stats, gfunc=np.sum, axis=0).unstack().unstack()

,g,max,mean,min,ptp
A,12.0,4.0,NaN,4.0,0.0
B,27.0,9.0,9.0,9.0,NaN
